## Preprocessing

In [224]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pickle

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [148]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
newapp_df=application_df.drop(['EIN','NAME'], axis=1)
newapp_df.head()

#Target variable is going to be whether or not the applicants applying for funding will be successful, as we can see in 
#the possible binary result of "1" or "0" in the "IS SUCCESSFUL" column. This is what we will be pulling out to to target
#for the model that we are going to create

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [27]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
#Print each column and what the unique values are for them
for col in newapp_df:
    print(newapp_df[col].unique())
    

    
    

#Print unique value counts   
n = newapp_df.nunique(axis=0)
 
print("No.of.unique values in each column :\n",
      n)

    
    

['T10' 'T3' 'T5' 'T7' 'T4' 'T6' 'T2' 'T9' 'T19' 'T8' 'T13' 'T12' 'T29'
 'T25' 'T14' 'T17' 'T15']
['Independent' 'CompanySponsored' 'Family/Parent' 'National' 'Regional'
 'Other']
['C1000' 'C2000' 'C3000' 'C1200' 'C2700' 'C7000' 'C7200' 'C1700' 'C4000'
 'C7100' 'C2800' 'C6000' 'C2100' 'C1238' 'C5000' 'C7120' 'C1800' 'C4100'
 'C1400' 'C1270' 'C2300' 'C8200' 'C1500' 'C7210' 'C1300' 'C1230' 'C1280'
 'C1240' 'C2710' 'C2561' 'C1250' 'C8000' 'C1245' 'C1260' 'C1235' 'C1720'
 'C1257' 'C4500' 'C2400' 'C8210' 'C1600' 'C1278' 'C1237' 'C4120' 'C2170'
 'C1728' 'C1732' 'C2380' 'C1283' 'C1570' 'C2500' 'C1267' 'C3700' 'C1580'
 'C2570' 'C1256' 'C1236' 'C1234' 'C1246' 'C2190' 'C4200' 'C0' 'C3200'
 'C5200' 'C1370' 'C2600' 'C1248' 'C6100' 'C1820' 'C1900' 'C2150']
['ProductDev' 'Preservation' 'Heathcare' 'CommunityServ' 'Other']
['Association' 'Co-operative' 'Trust' 'Corporation']
[1 0]
['0' '1-9999' '10000-24999' '100000-499999' '10M-50M' '25000-99999' '50M+'
 '1M-5M' '5M-10M']
['N' 'Y']
[    5000   108590

In [29]:
newapp_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [34]:
# Look at APPLICATION_TYPE value counts for binning
newapp_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [152]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# application_types_to_replace=[0,156,528,725,737,1065,1173, 1216,1542, 27037]        

# # Replace in dataframe
# for app in application_types_to_replace:
#     application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# # Check to make sure binning was successful
# application_df['APPLICATION_TYPE'].value_counts()

#Code obtained from RG who got it from Bill
newapp_df = newapp_df.apply(lambda x: x.mask(x.map(x.value_counts())<528, 'Other') if x.name=='APPLICATION_TYPE' else x)
newapp_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [116]:
#Check value counts for different instances of Classification
list_classification=newapp_df['CLASSIFICATION'].value_counts()
list_class=list_classification.tolist()
list_class

[17326,
 6074,
 4837,
 1918,
 1883,
 777,
 287,
 194,
 116,
 114,
 104,
 95,
 75,
 58,
 50,
 36,
 34,
 32,
 32,
 30,
 20,
 18,
 16,
 15,
 15,
 14,
 11,
 10,
 10,
 9,
 9,
 7,
 6,
 6,
 6,
 5,
 5,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [149]:
# Choose a cutoff value and create a list of classifications to be replaced
# # use the variable name `classifications_to_replace`
# #  YOUR CODE GOES HERE

# # Replace in dataframe
# for cls in classifications_to_replace:
#     application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# # Check to make sure binning was successful
# application_df['CLASSIFICATION'].value_counts()
newapp_df = newapp_df.apply(lambda x: x.mask(x.map(x.value_counts())<180, 'Other') if x.name=='CLASSIFICATION' else x)
newapp_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [157]:
# view newapp_df with new binned categories
newapp_df['APPLICATION_TYPE'].unique()
newapp_df['CLASSIFICATION'].unique()
newapp_df


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [193]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df=pd.get_dummies(newapp_df)
dummy_df.columns


Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C1700', 'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100',
       'CLASSIFICATION_C3000', 'CLASSIFICATION_C4000', 'CLASSIFICATION_C7000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-249

In [176]:
# Drop Special_Considerations_N column because it tells us the same thing as Special_Considerations_Y
# A value of 1 in Special_Considerations_Y column means YES
newdummy_df=dummy_df.drop(['SPECIAL_CONSIDERATIONS_N'], axis=1)
newdummy_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
34295,1,5000,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
34296,1,5000,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [185]:
# Split our preprocessed data into our features and target arrays
y=newdummy_df['IS_SUCCESSFUL'].values
X=newdummy_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [186]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [216]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=45))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 5)                 230       
                                                                 
 dense_12 (Dense)            (None, 1)                 6         
                                                                 
Total params: 236
Trainable params: 236
Non-trainable params: 0
_________________________________________________________________


In [217]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [221]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7310
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7319
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7316
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5480 - accuracy: 0.7323
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7317
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5478 - accuracy: 0.7326
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7322
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7311
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7328
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7322
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7322
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7329
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7328
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7332
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7336
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7329
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7338
Epoch 89/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7332
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 

In [222]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5526 - accuracy: 0.7259 - 572ms/epoch - 2ms/step
Loss: 0.552592933177948, Accuracy: 0.7259474992752075


In [225]:
# Export our model to HDF5 file
filename='AlphabetSoupCharity.h5'
pickle.dump(fit_model,open(filename,'wb'))

INFO:tensorflow:Assets written to: ram://ed9dbf15-a675-41d3-9154-d9f03883395d/assets
